## kappa系数简介

Kappa系数用于一致性检验，也可以用于衡量分类精度，kappa系数的计算是基于混淆矩阵的。 ——from百度百科
kappa系数取值为-1到1之间

## 基于混淆矩阵，手工计算kappa值

In [6]:
import numpy as np

def kappa(confusion_matrix):
    """计算kappa值系数"""
    sum_row_list = np.sum(confusion_matrix, axis=0)
    sum_col_list = np.sum(confusion_matrix, axis=1)
    sum_total = sum(col_sum)
    pe = np.dot(sum_row_list, sum_col_list) / float(sum_total ** 2)
    po = np.trace(confusion_matrix) / float(sum_total)
    return (po - pe) / (1 - pe)

In [ ]:
# 测试
confusion_matrix = np.array(
    [
        [21, 3, 19, 6, 81, 13, 12, 1, 0],
        [0, 47, 2, ],
        [],
        [],
        [],
        [],
        [],
        [],
    ]
)